# Artificial Intelligence Engineer Nanodegree
## Sudoku
----


## Setting up the Board
### Boxes, Units and Peers
And let's start naming the important elements created by these rows and columns that are relevant to solving a Sudoku:

- The individual squares at the intersection of rows and columns will be called **boxes**. These boxes will have labels 'A1', 'A2', ..., 'I9'.
- The complete rows, columns, and 3x3 squares, will be called **units**. Thus, each unit is a set of 9 boxes, and there are 27 units in total.
- For a particular box (such as 'A1'), its **peers** will be all other boxes that belong to a common unit (namely, those that belong to the same row, column, or 3x3 square).

Let's see an example. In the grids below, the set of highlighted boxes represent **units**. Each grid shows a different **peer** of the box at E3.

![sudoku](images/sudoku_1.png)

> Let's test your understanding! For any box, how many peers are there?

*Answer* : there are 20 boxes, as it can be seen in this example: Peers of 'A1': row: A2, A3, A4, A5, A6, A7, A8, A9 column: B1, C1, D1, E1, F1, G1, H1, I1 3x3 square: B2, B3, C2, C3 (since A1, A2, A3, B1, C1 are already counted).

## Encoding the Board

In [1]:
#각각의 하나의 셀을 박스(box)라 한다. 각 박스의 레이블은 열과 행 레이블을 조합해 'A1', 'A2' 등으로 부른다.
#숫자 나열의 단위가 되는 각 행, 열 그리고 3x3 사각형을 유닛(unit)라 한다. 따라서 각 유닛은 9개의 박스로 구성되고 총 27개의 유닛들이 있다.
#특정 박스가 속해있는 유닛의 다른 박스들을 피어(peer)라 한다. 따라서 피어는 특정 박스와 행, 열 또는 사각형 중 하나 이상(1~2)이 같아야 한다.

def cross(a, b): #각 문자열 a, b를 인자로 받아서 그 문자열의 각 요소로 만들어 낼 수 있는 모든 박스 리스트 반환
    return [s+t for s in a for t in b]

In [2]:
cross('abc', 'def') #cross 함수 예시.

['ad', 'ae', 'af', 'bd', 'be', 'bf', 'cd', 'ce', 'cf']

In [3]:
rows = "ABCDEFGHI"
cols = "123456789"

boxes = cross(rows, cols) #모든 가능한 박스 리스트

row_units = [cross(r, cols) for r in rows] #각 행 단위 리스트
#ex :: row_units[0] = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9']

column_units = [cross(rows, c) for c in cols] #각 열 단위 리스트
#ex :: column_units[0] = ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1']

square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
#square_units = [cross(rows[r:r+3], cols[c:c+3]) for r in range(0, len(rows), 3) for c in range(0, len(cols), 3)] #각 스퀘어 단위 리스트
#-> 이거 오류
#ex :: square_units[0] = ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']

unitlist = row_units + column_units + square_units #유닛들을 모두 하나의 list로
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
#units = {s: [u for u in unitlist if s in u] for s in boxes}
#박스 리스트에서 하나씩 가져와서 해당 박스가 유닛리스트 루프에 포함되면 해당 유닛을 추가한다.

peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)
#sum(units[s],[]) : 각 리스트들을 합친다.


In [4]:
def grid_values(grid): #문자열 형식을 딕셔너리로 변환
    """
    Convert grid string into {<box>: <value>} dict with '.' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '.' if it is empty.
    """
    
    assert len(grid) == 81, "Input grid must be a string of length 81 (9x9)"
    return dict(zip(boxes, grid)) #zip으로 합쳐서 딕셔너리로 생성



#     grid_list = list(grid)
#     return {box: grid_list.pop(0) for box in boxes}

In [5]:
def display(values): #문자열을 받아 스도쿠 모양으로 출력
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '') #center로 문자열 가운데 정렬
                      for c in cols))
        if r in 'CF': print(line)
    return

display(grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'))

. . 3 |. 2 . |6 . . 
9 . . |3 . 5 |. . 1 
. . 1 |8 . 6 |4 . . 
------+------+------
. . 8 |1 . 2 |9 . . 
7 . . |. . . |. . 8 
. . 6 |7 . 8 |2 . . 
------+------+------
. . 2 |6 . 9 |5 . . 
8 . . |2 . 3 |. . 9 
. . 5 |. 1 . |3 . . 


## Strategy 1: Elimination

If a box has a value assigned, then none of the peers of this box can have this value.

In [6]:
def grid_values(grid): #문자열 형식을 딕셔너리로 변환. 위의 grid_values와 달리 빈 박스의 경우 123456789를 반환한다.
    """
    Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    values = []
    all_digits = "123456789"
    
    for c in grid:
        if c == '.':
            values.append(all_digits)
        elif c in all_digits:
            values.append(c)
            
    assert len(values) == 81
    return dict(zip(boxes, values))

In [7]:
display(grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'))

123456789 123456789     3     |123456789     2     123456789 |    6     123456789 123456789 
    9     123456789 123456789 |    3     123456789     5     |123456789 123456789     1     
123456789 123456789     1     |    8     123456789     6     |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     8     |    1     123456789     2     |    9     123456789 123456789 
    7     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789     8     
123456789 123456789     6     |    7     123456789     8     |    2     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     2     |    6     123456789     9     |    5     123456789 123456789 
    8     123456789 123456789 |    2     123456789     3     |123456789 123456789     9     
123456789 123456789     5     |123456789     1     123456789 |    3   

In [8]:
def eliminate(values):
    """
    Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit,'')
            
    return values



#     all_digits = "123456789"
#     return_dic = {box: all_digits for box in boxes}

#     for key, value in values.items():
#         if value is all_digits:
#             for peer in peers[key]:
#                 if values[peer] is not all_digits:
#                     return_dic[key] = return_dic[key].replace(values[peer], "")
#         else:
#             return_dic[key] = values[key]
    
#     return return_dic



#     all_digits = "123456789"
#     return_dic = {box: all_digits for box in boxes}

#     for key, value in values.items():
#         """# peer 구하는 과정 """
#         if value is all_digits:
#             row_index = rows.index(key[0])
#             col_index = cols.index(key[-1])
#             squre_index = 3*(int(row_index)//3) + int(col_index)//3

#             unit_list = sorted(list(set(row_units[row_index] + column_units[col_index] + square_units[squre_index]) - set([key])))
#         """# peer 구하는 과정"""

#             for kk in unit_list:
#                 if values[kk] is not all_digits:
#                     return_dic[key] = return_dic[key].replace(values[kk], "")
#         else:
#             return_dic[key] = values[key]
    
#     return return_dic

In [9]:
display(eliminate(grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..')))

   45    4578    3   |   49     2     147  |   6     5789    57  
   9    24678    47  |   3      47     5   |   78    278     1   
   25    257     1   |   8      79     6   |   4    23579   2357 
---------------------+---------------------+---------------------
  345    345     8   |   1     3456    2   |   9    34567  34567 
   7    123459   49  |  459   34569    4   |   1    13456    8   
  1345  13459    6   |   7     3459    8   |   2     1345   345  
---------------------+---------------------+---------------------
  134    1347    2   |   6     478     9   |   5     1478    47  
   8     1467    47  |   2     457     3   |   17    1467    9   
   46    4679    5   |   4      1      47  |   3    24678   2467 


## Strategy 2: Only Choice

![sudoku_2.png](images/sudoku_2.png)

In [10]:
def only_choice(values):
    """
    Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.

    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    """
    # TODO: Implement only choice strategy here
    for unit in unitlist: #각각의 유닛 불러오기
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]] #위 유닛의 각 요소 레이블에서 digit 숫자가 그 값에 있다면 레이블 저장
            #숫자를 포함하고 있는 박스의 수
            if len(dplaces) == 1: #유일한 값을 찾아낸다.
                values[dplaces[0]] = digit
    
    return values

In [11]:
display(only_choice(eliminate(grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'))))

  45    8     3   |  9     2     1   |  6    5789   57  
  9     6     47  |  3     4     5   |  8    278    1   
  2    257    1   |  8     7     6   |  4   23579  2357 
------------------+------------------+------------------
 345   345    8   |  1    3456   2   |  9   34567 34567 
  7     2     9   |  5   34569   4   |  1   13456   8   
 1345 13459   6   |  7    3459   8   |  2    1345  345  
------------------+------------------+------------------
 134   1347   2   |  6     8     9   |  5    1478   47  
  8    1467   47  |  2     5     3   |  17    6     9   
  6     9     5   |  4     1     7   |  3     8     2   


## Constraint Propagation
제약 조건을 전파하여 점점 더 간결하게 만든다. 여기서는 Elimination과 Only Choice이 제약조건

In [12]:
def reduce_puzzle(values):
    """
    Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1]) #해결한 박스의 수

        # Your code here: Use the Eliminate Strategy
        values = eliminate(values)

        # Your code here: Use the Only Choice Strategy
        values = only_choice(values)

        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1]) #해결한 박스 수
        
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after #해결한 것이 하나도 없다면 멈춘다.
        
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]): #0인 값이 있는 지 확인
            return False
            #어떤 시점에서 사용 가능한 값이없는 상자가 있으면 False를 반환
        
    return values

In [13]:
display(reduce_puzzle(grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..')))

4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 


## Strategy 3: Search
복잡한 스도쿠 모형에선 위의 전략이 통하지 않을 때가 있다. 기본 로직으로 풀리지 않는 경우, 주어진 경우의 수를 대입해 가다보면 풀 수 있지만, 그것이 구현되지 않은 것. 가장 경우의 수가 적은 박스부터 풀어나가면 더 효율적

In [14]:
grid2 = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
values = grid_values(grid2)
display(reduce_puzzle(values))

   4      1679   12679  |  139     2369    269   |   8      1239     5    
 26789     3    1256789 | 14589   24569   245689 | 12679    1249   124679 
  2689   15689   125689 |   7     234569  245689 | 12369   12349   123469 
------------------------+------------------------+------------------------
  3789     2     15789  |  3459   34579    4579  | 13579     6     13789  
  3679   15679   15679  |  359      8     25679  |   4     12359   12379  
 36789     4     56789  |  359      1     25679  | 23579   23589   23789  
------------------------+------------------------+------------------------
  289      89     289   |   6      459      3    |  1259     7     12489  
   5      6789     3    |   2      479      1    |   69     489     4689  
   1      6789     4    |  589     579     5789  | 23569   23589   23689  


![sudoku_3.png](images/sudoku_3.png)

![sudoku_4.png](images/sudoku_4.png)

In [15]:
def search(values):
    "Using depth-first search and propagation, create a search tree and solve the sudoku."
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    
    if values is False: #어떤 시점에서 사용 가능한 값이없는 상자가 있어 False로 종료된 경우.
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in boxes): #모든 문제 해결했으면 바로 반환
        #any()는 Iterational 객체를 인자로 받아 어느 하나라도 True일때 True 반환
        #all()은 Iterational 객체를 인자로 받아 모두 True일때 True 반환
        return values ## Solved!
    
    # Choose one of the unfilled squares with the fewest possibilities
    n,s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1) #해결되지 않은 박스 중 가장 경우의 수가 적은 것을 택한다.
    # Now use recursion to solve each one of the resulting sudokus, and if one returns a value (not False), return that answer!
    for value in values[s]: #해결되지 않은 것의 가능한 경우의 수
        new_sudoku = values.copy() #게임 복사
        new_sudoku[s] = value #임의의 값을 차례대로 대입해 푼다.
        attempt = search(new_sudoku) #다시 호출
        if attempt: #해결 됐을 경우 all(len(values[s]) == 1 for s in boxes) = True
            return attempt

In [16]:
grid1 = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
values = grid_values(grid1)
display(search(values))

4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 


In [17]:
grid2 = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
values = grid_values(grid2)
display(search(values))

4 1 7 |3 6 9 |8 2 5 
6 3 2 |1 5 8 |9 4 7 
9 5 8 |7 2 4 |3 1 6 
------+------+------
8 2 5 |4 3 7 |1 6 9 
7 9 1 |5 8 6 |4 3 2 
3 4 6 |9 1 2 |7 5 8 
------+------+------
2 8 9 |6 4 3 |5 7 1 
5 7 3 |2 9 1 |6 8 4 
1 6 4 |8 7 5 |2 9 3 


## Naked Twins

![sudoku_5.png](images/sudoku_5.png)

In [18]:
def naked_twins(values):
    """Eliminate values using the naked twins strategy.
    Args:
        values(dict): a dictionary of the form {'box_name': '123456789', ...}

    Returns:
        the values dictionary with the naked twins eliminated from peers.
    """
    for unit in unitlist:  
        # Find all instances of naked twins
        value_list = [values[box] for box in unit] #유닛의 값 리스트
        twin_tuple = [(index, value) for index, value in enumerate(value_list) if value_list.count(value) is 2] #중복 값이 2개 있는 박스의 인덱스와 값
        twin_index_list = [tup[0] for tup in twin_tuple]
        twin_value_set = set([tup[1] for tup in twin_tuple])
        
        # Eliminate the naked twins as possibilities for their peers
        for index, box in enumerate(unit): #유닛의 다른 박스에 해당 중복값들 제거
            if index not in twin_index_list: #아예 없으면
                for value in twin_value_set:
                    if len(value) is 2:
                        values[box] = values[box].replace(value[0], "")
                        values[box] = values[box].replace(value[1], "")
            elif index in twin_index_list: #존재하는 경우
                temp_set = twin_value_set.copy()
                temp_set.remove(values[box])
                for value in temp_set:
                    if len(value) is 2:
                        values[box] = values[box].replace(value[0], "")
                        values[box] = values[box].replace(value[1], "") 
                    
    return values

In [19]:
before_naked_twins_1 = {'I6': '4', 'H9': '3', 'I2': '6', 'E8': '1', 'H3': '5', 'H7': '8', 'I7': '1', 'I4': '8',
                            'H5': '6', 'F9': '7', 'G7': '6', 'G6': '3', 'G5': '2', 'E1': '8', 'G3': '1', 'G2': '8',
                            'G1': '7', 'I1': '23', 'C8': '5', 'I3': '23', 'E5': '347', 'I5': '5', 'C9': '1', 'G9': '5',
                            'G8': '4', 'A1': '1', 'A3': '4', 'A2': '237', 'A5': '9', 'A4': '2357', 'A7': '27',
                            'A6': '257', 'C3': '8', 'C2': '237', 'C1': '23', 'E6': '579', 'C7': '9', 'C6': '6',
                            'C5': '37', 'C4': '4', 'I9': '9', 'D8': '8', 'I8': '7', 'E4': '6', 'D9': '6', 'H8': '2',
                            'F6': '125', 'A9': '8', 'G4': '9', 'A8': '6', 'E7': '345', 'E3': '379', 'F1': '6',
                            'F2': '4', 'F3': '23', 'F4': '1235', 'F5': '8', 'E2': '37', 'F7': '35', 'F8': '9',
                            'D2': '1', 'H1': '4', 'H6': '17', 'H2': '9', 'H4': '17', 'D3': '2379', 'B4': '27',
                            'B5': '1', 'B6': '8', 'B7': '27', 'E9': '2', 'B1': '9', 'B2': '5', 'B3': '6', 'D6': '279',
                            'D7': '34', 'D4': '237', 'D5': '347', 'B8': '3', 'B9': '4', 'D1': '5'}
display(before_naked_twins_1)

  1   237   4  | 2357  9   257 |  27   6    8  
  9    5    6  |  27   1    8  |  27   3    4  
  23  237   8  |  4    37   6  |  9    5    1  
---------------+---------------+---------------
  5    1   2379| 237  347  279 |  34   8    6  
  8    37  379 |  6   347  579 | 345   1    2  
  6    4    23 | 1235  8   125 |  35   9    7  
---------------+---------------+---------------
  7    8    1  |  9    2    3  |  6    4    5  
  4    9    5  |  17   6    17 |  8    2    3  
  23   6    23 |  8    5    4  |  1    7    9  


In [20]:
display(naked_twins(before_naked_twins_1))

  1   237   4  | 2357  9   257 |  27   6    8  
  9    5    6  |  27   1    8  |  27   3    4  
  23  237   8  |  4    37   6  |  9    5    1  
---------------+---------------+---------------
  5    1    79 | 237  347  279 |  34   8    6  
  8    3    79 |  6   347  579 | 345   1    2  
  6    4    23 | 1235  8   125 |  35   9    7  
---------------+---------------+---------------
  7    8    1  |  9    2    3  |  6    4    5  
  4    9    5  |  17   6    17 |  8    2    3  
  23   6    23 |  8    5    4  |  1    7    9  


## Diagonal Sudoku
대각선 유닛을 추가해 줘서 일반 유닛 외에 대각선으로도 스도쿠 조건에 만족하도록 한다.

In [21]:
cross_unit = [[rows[i] + cols[i] for i in range(len(rows))],
        [rows[i] + cols[len(rows)-1-i] for i in range(len(rows))]]
unitlist +=  cross_unit

In [22]:
grid1 = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
values = grid_values(grid1)
display(search(values))

2 6 7 |9 4 5 |3 8 1 
8 5 3 |7 1 6 |2 4 9 
4 9 1 |8 2 3 |5 7 6 
------+------+------
5 7 6 |4 3 8 |1 9 2 
3 8 4 |1 9 2 |6 5 7 
1 2 9 |6 5 7 |4 3 8 
------+------+------
6 4 2 |3 7 9 |8 1 5 
9 3 5 |2 8 1 |7 6 4 
7 1 8 |5 6 4 |9 2 3 


## Conclusion
각 각 전략을 추가하면서 최적화 해 나간다.

In [23]:
def reduce_puzzle(values):
    """
    Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1]) #해결한 박스의 수

        # Your code here: Use the Eliminate Strategy
        values = eliminate(values)

        # Your code here: Use the Only Choice Strategy
        values = only_choice(values)
        
        # Your code here: Use the Naked Twins Strategy
        values = naked_twins(values) #추가

        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1]) #해결한 박스 수
        
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after #해결한 것이 하나도 없다면 멈춘다.
        
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]): #0인 값이 있는 지 확인
            return False
            #어떤 시점에서 사용 가능한 값이없는 상자가 있으면 False를 반환
        
    return values

In [24]:
grid1 = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
values = grid_values(grid1)
display(search(values))

2 6 7 |9 4 5 |3 8 1 
8 5 3 |7 1 6 |2 4 9 
4 9 1 |8 2 3 |5 7 6 
------+------+------
5 7 6 |4 3 8 |1 9 2 
3 8 4 |1 9 2 |6 5 7 
1 2 9 |6 5 7 |4 3 8 
------+------+------
6 4 2 |3 7 9 |8 1 5 
9 3 5 |2 8 1 |7 6 4 
7 1 8 |5 6 4 |9 2 3 


# Question 1 (Naked Twins)
Q: How do we use constraint propagation to solve the naked twins problem?  
A: In my case, first, get units from *'unitlist'*.

# Question 2 (Diagonal Sudoku)
Q: How do we use constraint propagation to solve the diagonal sudoku problem?  
A: *Student should provide answer here*